In [ ]:
!pip install spacy

In [ ]:
!pip install https://huggingface.co/opennyaiorg/en_legal_ner_sm/resolve/main/en_legal_ner_sm-any-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.1
    Uninstalling 

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/ILDC_multi.csv'
df = pd.read_csv(file_path)

Mounted at /content/drive


In [ ]:
df

,text,label,split,name
0,"Uday Umesh Lalit, J. These appeals arise out ...",0,train,2020_1.txt
1,"Indira Banerjee, J. These appeals are against...",0,train,2020_2.txt
2,TABLE OF CONTENTS Introduction A Contentions B...,0,train,2020_3.txt
3,"Dinesh Maheshwari, J. Introductory with brief...",0,train,2020_4.txt
4,"Dinesh Maheshwari, J. Preliminary By way of t...",0,train,2020_5.txt
...,...,...,...,...
34811,"Dr. B. S. CHAUHAN, J. This appeal has been pr...",1,test,2013_101.txt
34812,"Rajendra Babu, J. The respondents, who are co...",0,test,2000_1559.txt
34813,Leave granted. Respondent 1 Insurance Company...,1,test,1998_83.txt
34814,Leave granted. The respondent is an Executive...,1,test,1995_2.txt


In [ ]:
df.drop(df.columns.difference(['text']), axis=1, inplace=True)

In [ ]:
df.info


<bound method DataFrame.info of                                                     text
0       Uday Umesh Lalit, J. These appeals arise out ...
1       Indira Banerjee, J. These appeals are against...
2      TABLE OF CONTENTS Introduction A Contentions B...
3       Dinesh Maheshwari, J. Introductory with brief...
4       Dinesh Maheshwari, J. Preliminary By way of t...
...                                                  ...
34811   Dr. B. S. CHAUHAN, J. This appeal has been pr...
34812   Rajendra Babu, J. The respondents, who are co...
34813   Leave granted. Respondent 1 Insurance Company...
34814   Leave granted. The respondent is an Executive...
34815   A. BOBDE, J. This appeal is directed against ...

[34816 rows x 1 columns]>

In [ ]:
df['text_id'] = range(1, len(df) + 1)

In [ ]:
df


,text,text_id
0,"Uday Umesh Lalit, J. These appeals arise out ...",1
1,"Indira Banerjee, J. These appeals are against...",2
2,TABLE OF CONTENTS Introduction A Contentions B...,3
3,"Dinesh Maheshwari, J. Introductory with brief...",4
4,"Dinesh Maheshwari, J. Preliminary By way of t...",5
...,...,...
34811,"Dr. B. S. CHAUHAN, J. This appeal has been pr...",34812
34812,"Rajendra Babu, J. The respondents, who are co...",34813
34813,Leave granted. Respondent 1 Insurance Company...,34814
34814,Leave granted. The respondent is an Executive...,34815


In [ ]:
import numpy as np
import pandas as pd

indices = np.array_split(df.index, 5)

df1 = df.loc[indices[0]]
df2 = df.loc[indices[1]]
df3 = df.loc[indices[2]]
df4 = df.loc[indices[3]]
df5 = df.loc[indices[4]]

In [ ]:
df1

,text,text_id
0,"Uday Umesh Lalit, J. These appeals arise out ...",1
1,"Indira Banerjee, J. These appeals are against...",2
2,TABLE OF CONTENTS Introduction A Contentions B...,3
3,"Dinesh Maheshwari, J. Introductory with brief...",4
4,"Dinesh Maheshwari, J. Preliminary By way of t...",5
...,...,...
6959,"Dr. ARIJIT PASAYAT, J. Challenge in this appe...",6960
6960,"TARUN CHATTERJEE,J. This appeal by special le...",6961
6961,"Dr. ARIJIT PASAYAT, J Leave granted. Challeng...",6962
6962,"B. Sinha, J. Leave granted. Application of Cl...",6963


In [ ]:
import spacy
import pandas as pd
import re

nlp = spacy.load("en_legal_ner_sm")

def apply_ner(row):
    text_id = row['text_id']
    text = row['text']

    chunk_size = 50000
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    result_df1 = []

    for chunk in chunks:
        doc = nlp(chunk)
        for ent in doc.ents:
            result_df1.append({'text_id': text_id, 'text': ent.text, 'label': ent.label_})

    return result_df1


result_df1_list = [apply_ner(row) for index, row in df1.iterrows()]

flattened_result = [item for sublist in result_df1_list for item in sublist]


combined_entities = []
i = 0
while i < len(flattened_result):
    current_entity = flattened_result[i]
    if (current_entity['label'] == 'PROVISION' and
        i + 1 < len(flattened_result) and
        flattened_result[i + 1]['label'] == 'STATUTE'):

        statute_text = re.sub(r'\b\d{4}\b', '', flattened_result[i + 1]['text']).strip()
        combined_text = f"{current_entity['text']} {statute_text}"
        combined_entities.append({'text_id': current_entity['text_id'], 'text': combined_text, 'label': 'PROVISION_STATUTE'})
        i += 2
    else:
        combined_entities.append(current_entity)
        i += 1

result_df1 = pd.DataFrame(combined_entities)

In [ ]:
result_df1

,text_id,text,label
0,1,Uday Umesh Lalit,ORG
1,1,09.12.2015,DATE
2,1,S.T. No.192 of 2014,CASE_NUMBER
3,1,12.03.2014,DATE
4,1,Writ Petition No.20650 W of 2013,CASE_NUMBER
...,...,...,...
774711,6964,"Section 2 g U.P. Reorganisation Act,",PROVISION_STATUTE
774712,6964,"U.P. Reorganisation Act, 2000",STATUTE
774713,6964,SLP No. 8708 of 2006,CASE_NUMBER
774714,6964,SLP C No. 6451 of 2005,CASE_NUMBER


Do the same for df2, df3, df4 and df5, whose results will be saved in result_df 2, 3, 4 and 5. Only proceed to the next step after this is done.

In [ ]:
result_df = pd.concat([result_df1, result_df2, result_df3, result_df4, result_df5], ignore_index=True)

In [ ]:
result_df

,text_id,text,label
0,1,Uday Umesh Lalit,ORG
1,1,09.12.2015,DATE
2,1,S.T. No.192 of 2014,CASE_NUMBER
3,1,12.03.2014,DATE
4,1,Writ Petition No.20650 W of 2013,CASE_NUMBER
...,...,...,...
2969157,34816,Roshan Lal,OTHER_PERSON
2969158,34816,Kishan Lal,OTHER_PERSON
2969159,34816,Suresh Singhal,OTHER_PERSON
2969160,34816,Kishan Lal,OTHER_PERSON


In [ ]:
result_df_backup = result_df
result_df_backup

,text_id,text,label
0,1,Uday Umesh Lalit,ORG
1,1,09.12.2015,DATE
2,1,S.T. No.192 of 2014,CASE_NUMBER
3,1,12.03.2014,DATE
4,1,Writ Petition No.20650 W of 2013,CASE_NUMBER
...,...,...,...
2969157,34816,Roshan Lal,OTHER_PERSON
2969158,34816,Kishan Lal,OTHER_PERSON
2969159,34816,Suresh Singhal,OTHER_PERSON
2969160,34816,Kishan Lal,OTHER_PERSON


In [ ]:
unique_labels = result_df['label'].unique()

print(unique_labels)

['ORG' 'DATE' 'CASE_NUMBER' 'PROVISION_STATUTE' 'STATUTE' 'COURT'
 'PRECEDENT' 'GPE' 'PROVISION' 'OTHER_PERSON' 'RESPONDENT' 'LAWYER'
 'JUDGE' 'WITNESS' 'PETITIONER']


In [ ]:
labels_to_keep = ['PROVISION_STATUTE', 'STATUTE', 'COURT', 'PRECEDENT', 'PROVISION']

filtered_df = result_df[result_df['label'].isin(labels_to_keep)]

In [ ]:
filtered_df

,text_id,text,label
5,1,"Sections 8, 10, 11 and 12 West Bengal Madrasah...",PROVISION_STATUTE
6,1,Commission Act,STATUTE
7,1,High Court of Judicature at Calcutta,COURT
9,1,"Sections 8, 10, 11 and 12 Commission Act",PROVISION_STATUTE
10,1,Commission Act,STATUTE
...,...,...,...
2969127,34816,Penal Code,STATUTE
2969138,34816,Exception 2 to Section 300,PROVISION
2969140,34816,Section 300,PROVISION
2969148,34816,Section 304 IPC,PROVISION_STATUTE


In [ ]:
filtered_df.to_csv('ILDC_multi_concatinated_words.csv')
from google.colab import files
files.download('ILDC_multi_concatinated_words.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df = df[['text_id', 'text']]

In [ ]:
df

,text_id,text
0,1,"Uday Umesh Lalit, J. These appeals arise out ..."
1,2,"Indira Banerjee, J. These appeals are against..."
2,3,TABLE OF CONTENTS Introduction A Contentions B...
3,4,"Dinesh Maheshwari, J. Introductory with brief..."
4,5,"Dinesh Maheshwari, J. Preliminary By way of t..."
...,...,...
34811,34812,"Dr. B. S. CHAUHAN, J. This appeal has been pr..."
34812,34813,"Rajendra Babu, J. The respondents, who are co..."
34813,34814,Leave granted. Respondent 1 Insurance Company...
34814,34815,Leave granted. The respondent is an Executive...


In [ ]:
df.to_csv('ILDC_multi_text_id.csv')
from google.colab import files
files.download('ILDC_multi_text_id.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>